In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
np.random.seed(42)
df = pd.read_csv("40403863_features.csv")

In [6]:
# section 3.1

kfolds = 5
kf = KFold(n_splits=kfolds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(df):
    train_data = df.iloc[train_index]
    test_data = df.iloc[test_index]

    